### Negative Indexer


This semi-supervised approach seem to be working fine


** aJavaher: this is the working version.

In [1]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import time
import pandas as pd
import numpy as np
import re
from hatesonar import Sonar

In [2]:
analyzer = SentimentIntensityAnalyzer() 

In [3]:
# reading the dataset

data = pd.read_csv('../Data/data_IRA_Ads.csv', sep='\t') 

data['pos'] = np.nan
data['neg'] = np.nan
data['neu'] = np.nan
data['compound'] = np.nan
data['flag'] = np.nan

for index, eachAdText in data['AD_TEXT'].iteritems():
    # skip empty texts
    if pd.isnull(eachAdText):
        continue
        
    # calculate the senti-index for each advert text
    tempRespond = analyzer.polarity_scores(eachAdText)    
    
    data.loc[index, 'pos'] = tempRespond['pos']
    data.loc[index, 'neg'] = tempRespond['neg']
    data.loc[index, 'neu'] = tempRespond['neu']
    data.loc[index, 'compound'] = tempRespond['compound']
    
    # make the flags
    if 0.5 <= tempRespond['compound']  and tempRespond['compound'] <= 1:
        data.loc[index, 'flag'] = 'positive'
    elif -0.5 < tempRespond['compound']  and tempRespond['compound'] < 0.5:
        data.loc[index, 'flag'] = 'neutral'
    else:
        data.loc[index, 'flag'] = 'negative'
    
    

In [4]:
#print(data[['AD_ID', 'pos', 'neg', 'neu', 'compound']])  

In [5]:
# find classes of ads
tempClass = data.loc[ (data['compound'] >= -1) &  (data['compound'] <= -0.5)  ]
print ('- Ads:', len(tempClass))

# find classes of ads
tempClass = data.loc[ (data['compound'] > -0.5) &  (data['compound'] < 0.5)  ]
print ('~ Ads:', len(tempClass))

# find classes of ads
tempClass = data.loc[ (data['compound'] >= 0.5) &  (data['compound'] <= 1)  ]
print ('+ Ads:', len(tempClass))

- Ads: 871
~ Ads: 1414
+ Ads: 1202


In [8]:
# finalize column names
data.rename(columns={'pos':'positivity_index'}, inplace=True)
data.rename(columns={'neg':'negativity_index'}, inplace=True)
data.rename(columns={'neu':'neutrality_index'}, inplace=True)
data.rename(columns={'compound':'compound_measure'}, inplace=True)



# save the processed data 
data.to_csv('../Data/processed_data.csv', sep='\t', encoding='utf-8')